In [238]:
# Install packages
import numpy as np
import pandas as pd
import re
import os

# Read packages
ds_jobs_raw = pd.read_csv('../1_Data_extraction/Output/job_offers.csv')

In [239]:
# Initial cleaning to standardize format of each column

# Create a copy
ds_jobs = ds_jobs_raw.copy()

# job_title
ds_jobs['job_title'] = ds_jobs['job_title'].str.title()
ds_jobs['job_title'] = ds_jobs['job_title'].replace('\(.*\)', '', regex = True)

# company_name
ds_jobs['company_name'] = ds_jobs['company_name'].str.title()

# company_location
ds_jobs['company_location'] = ds_jobs['company_location'].str.title()
ds_jobs['company_location'] = ds_jobs['company_location'].str.split(',', expand = True)[0]

In [240]:
# post_date >> days_from_post_date
ds_jobs['post_date'] = ds_jobs['post_date'].str.replace(' ago', '', regex = True)
ds_jobs[['post_date_qty', 'post_date_unit']] = ds_jobs['post_date'].str.split(' ', 1, expand = True)
ds_jobs['post_date_unit'] = ds_jobs['post_date_unit'].str.replace('s', '', regex = True)
ds_jobs.loc[ds_jobs['post_date_unit']=="week", "post_date_unit"] = 7
ds_jobs.loc[ds_jobs['post_date_unit']=="month", "post_date_unit"] = 28
ds_jobs.loc[ds_jobs['post_date_unit']=="year", "post_date_unit"] = 365
ds_jobs.loc[ds_jobs['post_date_unit']=="day", "post_date_unit"] = 1
ds_jobs.loc[ds_jobs['post_date_unit']=="hour", "post_date_unit"] = 1 / ds_jobs.loc[ds_jobs['post_date_unit']=="hour", "post_date_qty"].astype(float)
ds_jobs['post_date'] = ds_jobs['post_date_qty'].astype(float) * ds_jobs['post_date_unit'].astype(float)
ds_jobs = ds_jobs.drop(["post_date_qty", "post_date_unit"], axis = 1)
ds_jobs = ds_jobs.rename(columns = {"post_date": "days_from_post_date"})
ds_jobs['days_from_post_date'] = ds_jobs['days_from_post_date'].astype(int)

# no_of_applicants
ds_jobs['no_of_applicants'] = ds_jobs['no_of_applicants'].str.replace(' applicants', '')
ds_jobs['no_of_applicants'] = ds_jobs['no_of_applicants'].str.replace(' applicant', '').astype(int)

In [241]:
# work_time >> work_type, seniority_level
ds_jobs['count_dots'] = ds_jobs['work_time'].str.count('·').astype(int)
ds_jobs.loc[ds_jobs['count_dots'] == 2, 'work_time'] = ds_jobs.loc[ds_jobs['count_dots'] == 2, 'work_time'].str.split('· ', 1, expand = True)[1]
ds_jobs.loc[ds_jobs['work_time'].str.contains(' - '), 'work_time'] = ds_jobs.loc[ds_jobs['work_time'].str.contains(' - '), 'work_time'].str.split('· ', 1, expand = True)[1]
ds_jobs[['work_type', 'seniority_level']] = ds_jobs['work_time'].str.split(' · ', expand = True)
ds_jobs = ds_jobs.drop(['work_time', 'count_dots'], axis = 1)
cols = ds_jobs.columns.to_list()
new_cols = cols[0:4] + cols[-2:] + cols[4:-2]
ds_jobs = ds_jobs[new_cols]

# no_of_employees
ds_jobs.loc[ds_jobs['no_of_employees'].str.contains('See', na = True),'no_of_employees'] = np.nan
ds_jobs.loc[ds_jobs['no_of_employees'].str.contains('IT', na = True),'no_of_employees'] = np.nan
ds_jobs['no_of_employees'] = ds_jobs['no_of_employees'].str.replace(' employees', '')
ds_jobs['no_of_employees'] = ds_jobs['no_of_employees'].str.replace(',', '')

In [242]:
# job_desc >> extract tools used, python libraries used
ds_jobs['job_desc'] = ds_jobs['job_desc'].str.lower()

ds_jobs['tool_python'] = ds_jobs['job_desc'].str.contains('python', na = True).astype(int)
ds_jobs['tool_r'] = ds_jobs['job_desc'].str.contains('r program|r language', na = True, regex=True).astype(int)
ds_jobs['tool_sql'] = ds_jobs['job_desc'].str.contains('sql|search query', na = True).astype(int)

ds_jobs['lib_numpy'] = ds_jobs['job_desc'].str.contains('numpy| np', na = True, regex=True).astype(int)
ds_jobs['lib_matplotlib'] = ds_jobs['job_desc'].str.contains('matplot|matplt', na = True, regex=True).astype(int)
ds_jobs['lib_pandas'] = ds_jobs['job_desc'].str.contains('panda', na = True).astype(int)
ds_jobs['lib_scipy'] = ds_jobs['job_desc'].str.contains('scipy', na = True).astype(int)
ds_jobs['lib_scikit'] = ds_jobs['job_desc'].str.contains('scikit', na = True).astype(int)
ds_jobs['lib_theano'] = ds_jobs['job_desc'].str.contains('theano', na = True).astype(int)
ds_jobs['lib_tensorflow'] = ds_jobs['job_desc'].str.contains('tensor', na = True).astype(int)
ds_jobs['lib_keras'] = ds_jobs['job_desc'].str.contains('keras', na = True).astype(int)
ds_jobs['lib_pytorch'] = ds_jobs['job_desc'].str.contains('pytorch', na = True).astype(int)
ds_jobs['lib_cntk'] = ds_jobs['job_desc'].str.contains('cntk', na = True).astype(int)
ds_jobs['lib_mxnet'] = ds_jobs['job_desc'].str.contains('mxnet', na = True).astype(int)

In [243]:
# industry
industry_map = {'IT Services and IT Consulting':'Information Technology', 
              'Telecommunications':'Telecommunications',
              'Higher Education':'Education',
              'Software Development':'Information Technology', 
              'Investment Banking':'Financial Services',
              'Financial Services':'Financial Services', 
              'Insurance':'Financial Services', 
              'Entertainment Providers':'Entertainment and Media',
              'Security and Investigations':'Others', 
              'Accounting':'Financial Services', 
              'Oil and Gas':'Power',
              'Banking':'Financial Services', 
              'Human Resources':'Human Resources',
              'Technology, Information and Internet':'Information Technology',
              'Medical Equipment Manufacturing':'Healthcare', 
              'Government Administration':'Public Sector',
              'Staffing and Recruiting':'Human Resources',
              'Broadcast Media Production and Distribution':'Entertainment and Media',
              'Manufacturing':'Manufacturing',
              'Automation Machinery Manufacturing':'Manufacturing',
              'Business Consulting and Services':'Professional Services',
              'Transportation, Logistics, Supply Chain and Storage':'Transportation',
              'Renewable Energy Semiconductor Manufacturing':'Manufacturing',
              'Advertising Services':'Entertainment and Media', 
              'Semiconductor Manufacturing':'Manufacturing',
              'International Trade and Development':'Financial Services', 
              'Beverage Manufacturing':'Consumer goods',
              'Investment Management':'Financial Services', 
              'Food and Beverage Services':'Consumer goods',
              'Airlines and Aviation':'Transportation', 
              'Architecture and Planning':'Others',
              'Professional Services':'Professional Services', 
              'Human Resources Services':'Human Resources',
              'Biotechnology Research':'Healthcare', 
              'Industrial Machinery Manufacturing':'Manufacturing',
              'Information Technology & Services':'Information Technology', 
              'Semiconductors':'Manufacturing',
              'Computer Games':'Entertainment and Media', 
              'Research Services':'Others', 
              'Utilities':'Power', 
              'Law Practice':'Professional Services',
              'Appliances, Electrical, and Electronics Manufacturing':'Manufacturing',
              'Mechanical Or Industrial Engineering':'Manufacturing', 
              'Information Services':'Telecommunications',
              'Internet Publishing':'Others', 
              'Pharmaceutical Manufacturing':'Healthcare',
              'Motor Vehicle Manufacturing':'Manufacturing', 
              'Real Estate':'Real Estate',
              'Aviation and Aerospace Component Manufacturing':'Manufacturing', 
              'Hospitality':'Healthcare',
              'Computer and Network Security':'Information Technology', 
              'Plastics Manufacturing':'Manufacturing',
              'Personal Care Product Manufacturing':'Manufacturing', 
              'Retail':'Consumer goods',
              'Hospitals and Health Care':'Healthcare', 
              'Retail Apparel and Fashion':'Consumer goods',
              'Farming':'Agriculture', 
              'Venture Capital and Private Equity Principals':'Financial Services',
              'Retail Luxury Goods and Jewelry':'Consumer goods', 
              'Construction':'Others',
              'Computer Hardware Manufacturing':'Information Technology', 
              'Chemical Manufacturing':'Manufacturing',
              'Blockchain Services':'Financial Services', 
              'Mining':'Others', 
              'Design Services':'Others',
              'Gambling Facilities and Casinos':'Others',
              'Technology, Information and Media':'Entertainment and Media',
              'Food and Beverage Manufacturing':'Consumer goods', 
              'Civil Engineering':'Others',
              'Packaging and Containers Manufacturing':'Manufacturing', 
              'Education':'Education',
              'Public Relations and Communications Services':'Entertainment and Media', 
              'Entertainment':'Entertainment and Media',
              'Capital Markets':'Financial Services', 
              'Computers and Electronics Manufacturing':'Manufacturing',
              'Media Production':'Entertainment and Media', 
              'Machinery Manufacturing':'Manufacturing'}
ds_jobs['industry_new'] = ds_jobs['industry'].map(industry_map)

# Rearrange columns and remove job_desc column
cols = ds_jobs.columns.to_list()
new_cols = cols[0:10] + cols[-1:] + cols[11:-1]
ds_jobs = ds_jobs[new_cols]


In [244]:
# Export and save data
outname = 'job_offers_cleaned.csv'
outdir = './Output'
if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

ds_jobs.to_csv(fullname, index=False)